# This is an Notebook which can evalute multiple files.  It implements a number of features:

- TopK
- Removing recent referrals
- Thresholding 
- Tables and visualization
- Config files for standardized processes. 



In [298]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path

#path append
sys.path.append(os.path.join(Path.cwd(), 'modules'))
import Evaluate, Helper, Present, Score, Synthetic


<H3> Imports</H3>

### File configuration
<p> Just edit the configuration file path/address. prediction_files contain configs for different prediction files. config_file contains other configs for referral, visualization, experiment etc. </p>

In [299]:
config_file = 'config/config.yaml'
prediction_config_files = ['config/predictions/pred1.yaml', 'config/predictions/pred2.yaml']
generate_data=True


### Load Configuration and Referrals

In [300]:
'''
Transferred referral data loading inside the c_p for loop below, because referral_data is being changed for date format adjustment. 
As referral_data.copy() is not allowed for memory issues. The overhead shouldn't be much.
'''

#Load Configuration
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

#Generate Data (if required)
if generate_data:
    Synthetic.generate_synthetic_event_data(c_gen)
    Synthetic.generate_synthetic_prediction_data(c_gen)

### Evaluate Models
For each configuration file provided it will evaluate the models. 

In [301]:
for c_p in predictions:    
    prediction=Helper.read_file(directory=c_p['dir'],file=c_p['file'],file_format=c_p['file_format'],aws=c_p['aws'],bucket= c_p['bucket'])
    referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
    
    #TBD Implement a check that there is a match between prediction and evaluation file.
    
    all_model_evaluations = Evaluate.evaluate(c_p, c_e, c_r, referrals, prediction) 
    Present.present_evaluation(c_p, c_r, c_e, c_visual, all_model_evaluations)

Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR', 'lin_reg', 'rand_forest', 'xg_boost', 'sgmm']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Number of Unique IDs in Referral: 25
Number of Unique IDs in Prediction: 500
Number of Intersected IDs: 15


,Experiment Name,Model,Window,Eval Date,Num Samples,TN_@k=50,FP_@k=50,FN_@k=50,TP_@k=50,TN_@k=60,FP_@k=60,FN_@k=60,TP_@k=60,TN_@p>=0.5,FP_@p>=0.5,FN_@p>=0.5,TP_@p>=0.5,TN_@p>=0.6,FP_@p>=0.6,FN_@p>=0.6,TP_@p>=0.6,precision_@k=50,precision_@k=60,precision_@p>=0.5,precision_@p>=0.6,recall_@k=50,recall_@k=60,recall_@p>=0.5,recall_@p>=0.6,accuracy_@k=50,accuracy_@k=60,accuracy_@p>=0.5,accuracy_@p>=0.6,balanced_acc_@k=50,balanced_acc_@k=60,balanced_acc_@p>=0.5,balanced_acc_@p>=0.6,f1_score_@k=50,f1_score_@k=60,f1_score_@p>=0.5,f1_score_@p>=0.6,log_loss,roc_auc_score,brier_score_loss,Prediction Source,Referral Source,Result Output,Start Time,End Time,Total Time
0,Version 1.0,lin_reg,201701-201704,201701,495,441.00000,50.00000,4.00000,0.00000,432.00000,59.00000,3.00000,1.00000,235.00000,256.00000,3.00000,1.00000,298.00000,193.00000,3.00000,1.00000,0.00000,0.01667,0.00389,0.00515,0.00000,0.25000,0.25000,0.25000,0.00000,0.25000,0.25000,0.25000,0.44908,0.56492,0.36431,0.42846,0.00000,0.03125,0.00766,0.01010,1.01330,0.31237,0.33334,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:17.753429-04:00,2020-06-03 22:32:17.800290-04:00,00:00:00.046861
1,Version 1.0,lin_reg,201701-201707,201701,495,441.00000,50.00000,4.00000,0.00000,432.00000,59.00000,3.00000,1.00000,235.00000,256.00000,3.00000,1.00000,298.00000,193.00000,3.00000,1.00000,0.00000,0.01667,0.00389,0.00515,0.00000,0.25000,0.25000,0.25000,0.00000,0.25000,0.25000,0.25000,0.44908,0.56492,0.36431,0.42846,0.00000,0.03125,0.00766,0.01010,1.01330,0.31237,0.33334,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:17.803298-04:00,2020-06-03 22:32:17.820237-04:00,00:00:00.016939
2,Version 1.0,lin_reg,201701-201801,201701,495,438.00000,50.00000,7.00000,0.00000,429.00000,59.00000,6.00000,1.00000,233.00000,255.00000,5.00000,2.00000,296.00000,192.00000,5.00000,2.00000,0.00000,0.01667,0.00778,0.01031,0.00000,0.14286,0.28571,0.28571,0.00000,0.14286,0.28571,0.28571,0.44877,0.51098,0.38159,0.44614,0.00000,0.02985,0.01515,0.01990,1.01966,0.32216,0.33557,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:17.821239-04:00,2020-06-03 22:32:17.834200-04:00,00:00:00.012961
3,Version 1.0,rand_forest,201701-201704,201701,495,441.00000,50.00000,4.00000,0.00000,431.00000,60.00000,4.00000,0.00000,250.00000,241.00000,2.00000,2.00000,303.00000,188.00000,3.00000,1.00000,0.00000,0.00000,0.00823,0.00529,0.00000,0.00000,0.50000,0.25000,0.00000,0.00000,0.50000,0.25000,0.44908,0.43890,0.50458,0.43355,0.00000,0.00000,0.01619,0.01036,1.01271,0.42006,0.33323,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:17.835214-04:00,2020-06-03 22:32:17.850158-04:00,00:00:00.014944
4,Version 1.0,rand_forest,201701-201707,201701,495,441.00000,50.00000,4.00000,0.00000,431.00000,60.00000,4.00000,0.00000,250.00000,241.00000,2.00000,2.00000,303.00000,188.00000,3.00000,1.00000,0.00000,0.00000,0.00823,0.00529,0.00000,0.00000,0.50000,0.25000,0.00000,0.00000,0.50000,0.25000,0.44908,0.43890,0.50458,0.43355,0.00000,0.00000,0.01619,0.01036,1.01271,0.42006,0.33323,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:17.850158-04:00,2020-06-03 22:32:17.865117-04:00,00:00:00.014959
5,Version 1.0,rand_forest,201701-201801,201701,495,438.00000,50.00000,7.00000,0.00000,428.00000,60.00000,7.00000,0.00000,249.00000,239.00000,3.00000,4.00000,301.00000,187.00000,5.00000,2.00000,0.00000,0.00000,0.01646,0.01058,0.00000,0.00000,0.57143,0.28571,0.00000,0.00000,0.57143,0.28571,0.44877,0.43852,0.54084,0.45126,0.00000,0.00000,0.03200,0.02041,1.01261,0.46107,0.33318,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/refer

Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR', 'lin_reg', 'rand_forest', 'xg_boost', 'sgmm']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Number of Unique IDs in Referral: 25
Number of Unique IDs in Prediction: 500
Number of Intersected IDs: 15


,Experiment Name,Model,Window,Eval Date,Num Samples,TN_@k=50,FP_@k=50,FN_@k=50,TP_@k=50,TN_@k=60,FP_@k=60,FN_@k=60,TP_@k=60,TN_@p>=0.5,FP_@p>=0.5,FN_@p>=0.5,TP_@p>=0.5,TN_@p>=0.6,FP_@p>=0.6,FN_@p>=0.6,TP_@p>=0.6,precision_@k=50,precision_@k=60,precision_@p>=0.5,precision_@p>=0.6,recall_@k=50,recall_@k=60,recall_@p>=0.5,recall_@p>=0.6,accuracy_@k=50,accuracy_@k=60,accuracy_@p>=0.5,accuracy_@p>=0.6,balanced_acc_@k=50,balanced_acc_@k=60,balanced_acc_@p>=0.5,balanced_acc_@p>=0.6,f1_score_@k=50,f1_score_@k=60,f1_score_@p>=0.5,f1_score_@p>=0.6,log_loss,roc_auc_score,brier_score_loss,Prediction Source,Referral Source,Result Output,Start Time,End Time,Total Time
0,Version 1.0,lin_reg,201701-201704,201701,495,441.00000,50.00000,4.00000,0.00000,432.00000,59.00000,3.00000,1.00000,235.00000,256.00000,3.00000,1.00000,298.00000,193.00000,3.00000,1.00000,0.00000,0.01667,0.00389,0.00515,0.00000,0.25000,0.25000,0.25000,0.00000,0.25000,0.25000,0.25000,0.44908,0.56492,0.36431,0.42846,0.00000,0.03125,0.00766,0.01010,1.01330,0.31237,0.33334,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:18.772324-04:00,2020-06-03 22:32:18.827202-04:00,00:00:00.054878
1,Version 1.0,lin_reg,201701-201707,201701,495,441.00000,50.00000,4.00000,0.00000,432.00000,59.00000,3.00000,1.00000,235.00000,256.00000,3.00000,1.00000,298.00000,193.00000,3.00000,1.00000,0.00000,0.01667,0.00389,0.00515,0.00000,0.25000,0.25000,0.25000,0.00000,0.25000,0.25000,0.25000,0.44908,0.56492,0.36431,0.42846,0.00000,0.03125,0.00766,0.01010,1.01330,0.31237,0.33334,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:18.830167-04:00,2020-06-03 22:32:18.844149-04:00,00:00:00.013982
2,Version 1.0,lin_reg,201701-201801,201701,495,438.00000,50.00000,7.00000,0.00000,429.00000,59.00000,6.00000,1.00000,233.00000,255.00000,5.00000,2.00000,296.00000,192.00000,5.00000,2.00000,0.00000,0.01667,0.00778,0.01031,0.00000,0.14286,0.28571,0.28571,0.00000,0.14286,0.28571,0.28571,0.44877,0.51098,0.38159,0.44614,0.00000,0.02985,0.01515,0.01990,1.01966,0.32216,0.33557,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:18.845136-04:00,2020-06-03 22:32:18.860087-04:00,00:00:00.014951
3,Version 1.0,rand_forest,201701-201704,201701,495,441.00000,50.00000,4.00000,0.00000,431.00000,60.00000,4.00000,0.00000,250.00000,241.00000,2.00000,2.00000,303.00000,188.00000,3.00000,1.00000,0.00000,0.00000,0.00823,0.00529,0.00000,0.00000,0.50000,0.25000,0.00000,0.00000,0.50000,0.25000,0.44908,0.43890,0.50458,0.43355,0.00000,0.00000,0.01619,0.01036,1.01271,0.42006,0.33323,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:18.861085-04:00,2020-06-03 22:32:18.874073-04:00,00:00:00.012988
4,Version 1.0,rand_forest,201701-201707,201701,495,441.00000,50.00000,4.00000,0.00000,431.00000,60.00000,4.00000,0.00000,250.00000,241.00000,2.00000,2.00000,303.00000,188.00000,3.00000,1.00000,0.00000,0.00000,0.00823,0.00529,0.00000,0.00000,0.50000,0.25000,0.00000,0.00000,0.50000,0.25000,0.44908,0.43890,0.50458,0.43355,0.00000,0.00000,0.01619,0.01036,1.01271,0.42006,0.33323,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 22:32:18.875058-04:00,2020-06-03 22:32:18.892011-04:00,00:00:00.016953
5,Version 1.0,rand_forest,201701-201801,201701,495,438.00000,50.00000,7.00000,0.00000,428.00000,60.00000,7.00000,0.00000,249.00000,239.00000,3.00000,4.00000,301.00000,187.00000,5.00000,2.00000,0.00000,0.00000,0.01646,0.01058,0.00000,0.00000,0.57143,0.28571,0.00000,0.00000,0.57143,0.28571,0.44877,0.43852,0.54084,0.45126,0.00000,0.00000,0.03200,0.02041,1.01261,0.46107,0.33318,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data